In [1]:
import sys
import os

# Source: https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

import category_encoders as ce

from lusi.ecoc import SVMRandomInvariantsECOC
from lusi.types import InvariantTypes
from utils import run_experiment

In [2]:
df = pd.read_csv("../data/glass.csv", header=None)
df

,0,1,2,3,4,5,6,7,8,9
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0,1
...,...,...,...,...,...,...,...,...,...,...
209,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0,7
210,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0,7
211,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0,7
212,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0,7


In [3]:
labels = np.unique(df.iloc[:, -1].values)
label_mapping = {
    orig_label: new_label
    for orig_label, new_label in zip(labels, range(len(labels)))
}

In [4]:
df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: label_mapping[x])
np.unique(df.iloc[:, -1].values)

array([0, 1, 2, 3, 4, 5])

In [5]:
X, y = df.iloc[:, :-1].values, df.iloc[:, -1].values
X

array([[ 1.52101, 13.64   ,  4.49   , ...,  8.75   ,  0.     ,  0.     ],
       [ 1.51761, 13.89   ,  3.6    , ...,  7.83   ,  0.     ,  0.     ],
       [ 1.51618, 13.53   ,  3.55   , ...,  7.78   ,  0.     ,  0.     ],
       ...,
       [ 1.52065, 14.36   ,  0.     , ...,  8.44   ,  1.64   ,  0.     ],
       [ 1.51651, 14.38   ,  0.     , ...,  8.48   ,  1.57   ,  0.     ],
       [ 1.51711, 14.23   ,  0.     , ...,  8.62   ,  1.67   ,  0.     ]])

In [6]:
num_dimensions = X.shape[1]
num_classes = len(np.unique(y))

print("Num dimensions: ", num_dimensions)
print("Num classes: ", num_classes)

Num dimensions:  9
Num classes:  6


In [7]:
encoding = np.eye(num_classes)
encoding

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [8]:
# Define models
models = [
    ('Baseline', SVMRandomInvariantsECOC(encoding)),
    ('Vapnik', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.VAPNIK, tolerance=20)),
    ('Projections', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.PROJECTION, tolerance=20)),
    ('Hyperplanes', SVMRandomInvariantsECOC(encoding, invariant_type=InvariantTypes.HYPERPLANE, tolerance=20)),
]

In [9]:
# Define models parameters
models_parameters = [
    {
        'kernel': ['rbf'],
        'C': [0.01, 0.1, 1.0, 10.0],
        'gamma': [0.01, 0.1, 1.0, 'auto'],
        'num_invariants': [0],
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 2),
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 1),
    },
    {
        'kernel': ['rbf'],
        'C': [0.001, 0.01, 0.1, 1.0, 10.0],
        'gamma': [0.001, 0.01, 0.1, 1.0, 'auto'],
        'delta': [0.001, 0.01, 0.1, 1.0],
        'num_invariants': np.arange(1, num_dimensions + 1),
    }
]

In [10]:
# Define seeds and train sizes
seeds = [47, 4, 81]
train_sizes = [1.0, 0.5, 0.1]

In [11]:
results_df = run_experiment(X, y, seeds, train_sizes, models, models_parameters)
results_df.to_csv('results/glass.csv')

-------------------- EXPERIMENT 1 --------------------




------ Using 1.0 of the training data ------


Training Baseline model

Running experiment with random_state=47
Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        num_invariants=0, random_state=47)
Accuracy:  0.6511627906976745
Running experiment with random_state=4
Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        num_invariants=0, random_state=4)
Accuracy:  0.6511627906976745
Running experiment with random_state=81
Best estima

Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        invariant_type=<InvariantTypes.VAPNIK: 'VAPNIK'>,
                        num_invariants=6, random_state=81, tolerance=20)
Accuracy:  0.627906976744186



Training Projections model

Running experiment with random_state=47
Best estimator:  SVMRandomInvariantsECOC(C=0.1,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        num_invariants=1, random_state=47, tolerance=20)
Accuracy:  0.6046511627906976
Running experiment with random_state=4
Best estimator:  SVMRandomInvariantsECOC(C=0.1,
 

/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.01,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        gamma=1.0, num_invariants=0, random_state=4)
Accuracy:  0.5116279069767442
Running experiment with random_state=81
Best estimator:  SVMRandomInvariantsECOC(C=0.01,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        gamma=1.0, num_invariants=0, random_state=81)
Accuracy:  0.5116279069767442



Training Vapnik model

Running experiment with random_state=47


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


...

Best estimator:  SVMRandomInvariantsECOC(C=10.0,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        gamma=0.1,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        random_state=47, tolerance=20)
Accuracy:  0.6744186046511628
Running experiment with random_state=4


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.001,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        gamma=0.001,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=3, random_state=4, tolerance=20)
Accuracy:  0.5116279069767442
Running experiment with random_state=81


/home/volokin/Repositories/Master/TFM/LUSI/env/lib/python3.10/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


Best estimator:  SVMRandomInvariantsECOC(C=0.01,
                        encoding=array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]]),
                        gamma=0.001,
                        invariant_type=<InvariantTypes.HYPERPLANE: 'HYPERPLANE'>,
                        num_invariants=2, random_state=81, tolerance=20)
Accuracy:  0.4418604651162791



